In [103]:
import pandas as pd
import requests, os
from datetime import datetime

In [ ]:
url = "https://p2p.binance.com/bapi/c2c/v2/friendly/c2c/adv/search"
csv_filename = "data/B.csv"


# Utils
def listing(json, *objectives):
    if not json:
        return ""
    if not objectives:
        return "|".join(map(str, json))
    return "|".join(
        map(lambda x: ":".join(str(x[obj]) for obj in objectives if obj in x), json)
    )


payload = {
    "asset": "USDT",
    "fiat": "BOB",
    "tradeType": "",
    "page": 1,
    "rows": 10,
    "payTypes": [],
    "countries": [],
    "publisherType": None,
}

time = int(datetime.now().timestamp())

all_data = []
for trade_type in ["BUY", "SELL"]:
    payload["tradeType"] = trade_type
    payload["page"] = 1  # Reset page number for each trade type
    while True:
        response = requests.post(url, json=payload)
        data = response.json()

        if not data["data"]:
            break

        all_data.extend(data["data"])
        payload["page"] += 1

df = pd.DataFrame(
    [
        {
            "timestamp": time,
            "advNo": adv["adv"]["advNo"],
            "classify": adv["adv"]["classify"],
            "tradeType": adv["adv"]["tradeType"],
            "advStatus": adv["adv"]["advStatus"],
            "priceType": adv["adv"]["priceType"],
            "priceFloatingRatio": round(
                float(adv["adv"]["priceFloatingRatio"] or 0), 5
            ),
            "rateFloatingRatio": round(float(adv["adv"]["rateFloatingRatio"] or 0), 5),
            "currencyRate": round(float(adv["adv"]["currencyRate"] or 0), 5),
            "price": round(float(adv["adv"]["price"] or 0), 5),
            "initAmount": round(float(adv["adv"]["initAmount"] or 0), 5),
            "surplusAmount": round(float(adv["adv"]["surplusAmount"] or 0), 5),
            "tradableQuantity": round(float(adv["adv"]["tradableQuantity"] or 0), 5),
            "amountAfterEditing": round(
                float(adv["adv"]["amountAfterEditing"] or 0), 5
            ),
            "maxSingleTransAmount": round(
                float(adv["adv"]["maxSingleTransAmount"] or 0), 5
            ),
            "minSingleTransAmount": round(
                float(adv["adv"]["minSingleTransAmount"] or 0), 5
            ),
            "buyerKycLimit": adv["adv"]["buyerKycLimit"],
            "buyerRegDaysLimit": int(adv["adv"]["buyerRegDaysLimit"] or 0),
            "buyerBtcPositionLimit": adv["adv"]["buyerBtcPositionLimit"],
            "payTimeLimit": round(float(adv["adv"]["payTimeLimit"] or 0), 5),
            "tradeMethods": listing(
                adv["adv"]["tradeMethods"], "identifier", "payType"
            ),
            "takerAdditionalKycRequired": adv["adv"]["takerAdditionalKycRequired"],
            "isTradable": adv["adv"]["isTradable"],
            "dynamicMaxSingleTransAmount": round(
                float(adv["adv"]["dynamicMaxSingleTransAmount"] or 0), 5
            ),
            "minSingleTransQuantity": round(
                float(adv["adv"]["minSingleTransQuantity"] or 0), 5
            ),
            "maxSingleTransQuantity": round(
                float(adv["adv"]["maxSingleTransQuantity"] or 0), 5
            ),
            "dynamicMaxSingleTransQuantity": round(
                float(adv["adv"]["dynamicMaxSingleTransQuantity"] or 0), 5
            ),
            "commissionRate": round(float(adv["adv"]["commissionRate"] or 0), 5),
            "tradeMethodCommissionRates": listing(
                adv["adv"]["tradeMethodCommissionRates"],
                "paymentMethod",
                "commissionRate",
            ),
            "isSafePayment": adv["adv"]["isSafePayment"],
            "userNo": adv["advertiser"]["userNo"],
            "monthOrderCount": int(adv["advertiser"]["monthOrderCount"] or 0),
            "monthFinishRate": round(
                float(adv["advertiser"]["monthFinishRate"] or 0), 5
            ),
            "positiveRate": round(float(adv["advertiser"]["positiveRate"] or 0), 5),
            "userType": adv["advertiser"]["userType"],
            "userGrade": adv["advertiser"]["userGrade"],
            "userIdentity": adv["advertiser"]["userIdentity"],
            "badges": listing(adv["advertiser"]["badges"]),
            "isBlocked": adv["advertiser"]["isBlocked"],
            "activeTimeInSecond": adv["advertiser"]["activeTimeInSecond"],
        }
        for adv in all_data
    ]
)

df.to_csv(csv_filename, header=True)

# compar


In [104]:
tradeType = "BUY"

In [ ]:
A = pd.read_csv("data/A.csv")
A = A.query(f"tradeType == '{tradeType}'")
A = A.groupby(A["userNo"]).agg(
    timestamp=("timestamp", "first"),
    orders=("monthOrderCount", "first"),
    price=("price", "min"),
    quantity=("tradableQuantity", "sum"),
)
A

,timestamp,orders,price,quantity
userNo,,,,
461ae4417c244f6b9b9fd276ec0e933f,1729707233,105,10.44,4144.25
s02a6779cfc4c367fb148d6f9c7f5e7d1,1729707233,12,8.50,2000.00
s04cf29e6fc5f39cfb17ca66c8b249a7d,1729707233,561,10.08,35068.33
s05481bb8357d34829ce66ef5327074e3,1729707233,2,6.86,498.53
s081da581488434208ff86aa141e15729,1729707233,34,9.00,100.00
...,...,...,...,...
sf9ca1195743a32e8bc57725b893a9ed3,1729707233,3177,10.62,1883.00
sfaa449a0d9423434a03e94ea62ec20e5,1729707233,15,6.94,141.00
sfc94122c9a4639e690f4c9be0265406f,1729707233,139,10.58,1418.79


In [ ]:
B = pd.read_csv("data/B.csv")
B = B.query(f"tradeType == '{tradeType}'")
B = B.groupby(B["userNo"]).agg(
    timestamp=("timestamp", "first"),
    orders=("monthOrderCount", "first"),
    price=("price", "min"),
    quantity=("tradableQuantity", "sum"),
)
B

,timestamp,orders,price,quantity
userNo,,,,
s02a6779cfc4c367fb148d6f9c7f5e7d1,1729740009,12,8.50,2000.00
s05481bb8357d34829ce66ef5327074e3,1729740009,2,6.86,498.53
s081da581488434208ff86aa141e15729,1729740009,34,9.00,100.00
s0f381032c2ae3bfca0cc5567b7f85a4f,1729740009,2,6.98,3000.00
s0ffc3574980531e39b0d2f79d770c0b1,1729740009,10,7.80,9987.70
...,...,...,...,...
sf0fa3a493fca3e82a7a035c6c54a3249,1729740009,2,9.00,500.00
sf516233b2c023f79bc9dbe49b7fc4a1b,1729740009,21,6.78,5399.13
sf78fbc6f25b03067a141d9fef7854a92,1729740009,1,8.50,1100.00


In [ ]:
# Alinear los índices y columnas de ambos DataFrames
A, B = A.align(B, join="outer", axis=0, fill_value=0)

# Comparar los DataFrames
comparison = A.compare(B)
comparison

timestamp              orders        \
                                         self       other    self other   
userNo                                                                    
461ae4417c244f6b9b9fd276ec0e933f   1729707233           0   105.0   0.0   
s02a6779cfc4c367fb148d6f9c7f5e7d1  1729707233  1729740009     NaN   NaN   
s04cf29e6fc5f39cfb17ca66c8b249a7d  1729707233           0   561.0   0.0   
s05481bb8357d34829ce66ef5327074e3  1729707233  1729740009     NaN   NaN   
s081da581488434208ff86aa141e15729  1729707233  1729740009     NaN   NaN   
...                                       ...         ...     ...   ...   
sf9ca1195743a32e8bc57725b893a9ed3  1729707233           0  3177.0   0.0   
sfaa449a0d9423434a03e94ea62ec20e5  1729707233  1729740009    15.0  16.0   
sfc94122c9a4639e690f4c9be0265406f  1729707233           0   139.0   0.0   
sfe49d9d9d81b36ccb4f522cd521e6ff9  1729707233  1729740009    92.0  91.0   
sff70d221ca663613a9f50ac7d46ca121  1729707233           0     8.0   0.0   

                                   price        quantity        
                                    self other      self other  
userNo                                                          
461ae4417c244f6b9b9fd276ec0e933f   10.44   0.0   4144.25   0.0  
s02a6779cfc4c367fb148d6f9c7f5e7d1    NaN   NaN       NaN   NaN  
s04cf29e6fc5f39cfb17ca66c8b249a7d  10.08   0.0  35068.33   0.0  
s05481bb8357d34829ce66ef5327074e3    NaN   NaN       NaN   NaN  
s081da581488434208ff86aa141e15729    NaN   NaN       NaN   NaN  
...                                  ...   ...       ...   ...  
sf9ca1195743a32e8bc57725b893a9ed3  10.62   0.0   1883.00   0.0  
sfaa449a0d9423434a03e94ea62ec20e5    NaN   NaN       NaN   NaN  
sfc94122c9a4639e690f4c9be0265406f  10.58   0.0   1418.79   0.0  
sfe49d9d9d81b36ccb4f522cd521e6ff9    NaN   NaN       NaN   NaN  
sff70d221ca663613a9f50ac7d46ca121  10.19   0.0   1500.00   0.0  

[177 rows x 8 columns]

In [ ]:
general_difference = B - A
general_difference

,timestamp,orders,price,quantity
userNo,,,,
461ae4417c244f6b9b9fd276ec0e933f,-1729707233,-105,-10.44,-4144.25
s02a6779cfc4c367fb148d6f9c7f5e7d1,32776,0,0.00,0.00
s04cf29e6fc5f39cfb17ca66c8b249a7d,-1729707233,-561,-10.08,-35068.33
s05481bb8357d34829ce66ef5327074e3,32776,0,0.00,0.00
s081da581488434208ff86aa141e15729,32776,0,0.00,0.00
...,...,...,...,...
sf9ca1195743a32e8bc57725b893a9ed3,-1729707233,-3177,-10.62,-1883.00
sfaa449a0d9423434a03e94ea62ec20e5,32776,1,0.00,0.00
sfc94122c9a4639e690f4c9be0265406f,-1729707233,-139,-10.58,-1418.79


In [ ]:
# Definir un umbral para la diferencia de timestamp
umbral_alto = 1000000  # Puedes ajustar este valor según sea necesario

# Filtrar las filas con diferencia negativa o muy alta en timestamp
filtered_difference = general_difference[(general_difference["timestamp"] >= 0)]
filtered_difference

,timestamp,orders,price,quantity
userNo,,,,
s02a6779cfc4c367fb148d6f9c7f5e7d1,32776,0,0.0,0.0
s05481bb8357d34829ce66ef5327074e3,32776,0,0.0,0.0
s081da581488434208ff86aa141e15729,32776,0,0.0,0.0
s0f381032c2ae3bfca0cc5567b7f85a4f,32776,0,0.0,0.0
s0ffc3574980531e39b0d2f79d770c0b1,32776,-1,0.0,0.0
...,...,...,...,...
sf0fa3a493fca3e82a7a035c6c54a3249,32776,0,0.0,0.0
sf516233b2c023f79bc9dbe49b7fc4a1b,32776,0,0.0,0.0
sf78fbc6f25b03067a141d9fef7854a92,32776,0,0.0,0.0


In [ ]:
import pandas

df = pandas.read_csv("data/currency_exchange_rates.csv")
df["timestamp"].max()

1730305645.0

In [28]:
import requests
from lxml import html
import re


def get_from_xpath(url, *path_pattern):
    response = requests.get(url)
    tree = html.fromstring(response.content)
    matches = []

    for xpath, pattern in path_pattern:
        text = tree.xpath(xpath)
        for t in text:
            content = t.text_content()
            match = re.search(pattern, content)
            if match:
                if match.groups():
                    for group in match.groups():
                        matches.append(group)
    if matches:
        return matches
    else:
        raise Exception("No se puedo extraer información")

In [29]:
get_from_xpath(
    "https://bancounion.com.bo/",
    ("//footer//div[2]/div[1]/p", r"Compra BOB: (\d+\.\d+) / Venta BOB: (\d+\.\d+)"),
)

['6.85', '6.97']

In [ ]:
get_from_xpath(
    "https://www.bmsc.com.bo/",
    ("/html/body/div/div/div/div/div[3]/div[1]/div[1]/div/div[1]/div/div/div/div/div/div[1]/span", r"Compra: (\d+\.\d+)"),
    ("/html/body/div/div/div/div/div[3]/div[1]/div[1]/div/div[1]/div/div/div/div/div/div[2]/span", r"Venta: (\d+\.\d+)"),
)

In [32]:
get_from_xpath(
    "https://www.bisa.com/",
    ("/html/body/nav[2]/div/div[2]/div/div/b[1]", r"(\d+\.\d+)"),
    ("/html/body/nav[2]/div/div[2]/div/div/b[2]", r"(\d+\.\d+)"),
)

['6.85', '6.97']

In [33]:
get_from_xpath(
    "https://www.baneco.com.bo/",
    ("//*[@id='cotizacion']", r"  Venta: Bs(\d+\.\d+) - Compra: Bs(\d+\.\d+) ")
)

Exception: No se puedo extraer información